In [65]:
import numpy as np
import pandas as pd
from parse import parse
import pprint
from aocd import get_data
from heapq import heappop, heappush, heapify
from itertools import cycle

data = get_data(year=2018, day=15)

test = r"""#########
#G..G..G#
#.......#
#.......#
#G..E..G#
#.......#
#.......#
#G..G..G#
#########"""

In [99]:
class Node(object):
    def __init__(self, x, y, e=None):
        self.x = x
        self.y = y
        self.e = set() if e is None else e
        
    @property
    def pos(self):
        return self.y, self.x
        
    def __hash__(self):
        return hash(self.pos)
    
    def __lt__(self, other):
        return self.pos < other.pos
    
    def __gt__(self, other):
        return self.pos > other.pos
    
    def __eq__(self, other):
        return self.pos == other.pos
    
    def __repr__(self):
        return f"(x: {self.x}, y: {self.y}) Edge Count: {len(self.e)}"
    
    @property
    def sides(self):
        return {(self.x + i, self.y + j) for i, j in [(-1, 0), (1, 0), (0, -1), (0, 1)]}
    
    def attackable(self, faction):
        return {
            node for node in self.e
            if isinstance(node, Creature)
            and node.faction != faction
        }
    
    def move(self, other):
        self.x, other.x = other.x, self.x
        self.y, other.y = other.y, self.y
                    
    def non_creature_edges(self):
        return {node for node in self.e if not isinstance(node, Creature)}
        

class Creature(Node):
    def __init__(self, x, y, hp, ap, faction):
        super().__init__(x, y)
        self.hp = hp
        self.ap = ap
        self.faction = faction
    
    def __repr__(self):
        return f"{super().__repr__()} | {self.faction}: AP={self.ap} <{self.hp}>"
    
    def attack(self, enemy):
        enemy.hp -= self.ap
        if enemy.hp <= 0:
            enemy = Node(enemy.x, enemy.y, enemy.e)
            
        return enemy

class Graph(object):
    def __init__(self, nodes):
        self.nodes = nodes
        self.setup()
        
    def setup(self):
        self._nodes = {node.pos[::-1]: node for node in self.nodes}
        self.connect()
        
    def connect(self):
        for node in self.nodes:
            node.e = {*filter(None, map(self.get, node.sides))}
    
    def __repr__(self):
        return pprint.pformat(self.nodes)
    
    def __getitem__(self, item):
        return self._nodes[item]
    
    def get(self, item):
        return self._nodes.get(item)

def to_array(data):
    return np.array([[*x] for x in data.splitlines()])

def part1(data):

    a = to_array(data)

    c = (a == 'E') | (a == 'G')
    o = (a == '.')

    creatures = [Creature(x, y, 200, 3, a[y, x]) for y, x in zip(*np.where(c))]
    graph = Graph({*map(Node, *np.where(o))} | {*creatures})


    return creatures, graph

In [104]:
creatures, graph = part1(test)

someone_moved = True
round_number = 0
while someone_moved:
    someone_moved = False
    
    for creature in creatures:
        if creature.hp <= 0:
            continue

        faction = creature.faction
        attackable = creature.attackable(faction)
        if attackable:
            enemy = min(attackable, key=lambda c: (c.hp, c))
            graph.nodes.remove(enemy)
            graph.nodes.add(creature.attack(enemy))
            graph.connect()

            someone_moved = True

        else:
            seen = creature.e
            q = [(e,) for e in creature.e]
            heapify(q)

            while q:
                path = heappop(q)
                attackable = path[-1].attackable(faction)
                if attackable:
                    creature.move(path[0])
                    graph.connect()
                    someone_moved = True
                else:
                    nodes = path[-1].non_creature_edges() - seen
                    for node in nodes:
                        heappush(q, path + (node,))
                    seen |= nodes
                
            
    round_number += 1

KeyError: (x: 7, y: 3) Edge Count: 3 | E: AP=3 <200>

In [105]:
creatures

[(x: 2, y: 1) Edge Count: 3 | G: AP=3 <200>,
 (x: 3, y: 1) Edge Count: 3 | G: AP=3 <200>,
 (x: 6, y: 1) Edge Count: 3 | G: AP=3 <200>,
 (x: 1, y: 3) Edge Count: 3 | G: AP=3 <200>,
 (x: 7, y: 3) Edge Count: 3 | E: AP=3 <200>,
 (x: 5, y: 4) Edge Count: 4 | G: AP=3 <200>,
 (x: 1, y: 5) Edge Count: 3 | G: AP=3 <200>,
 (x: 4, y: 5) Edge Count: 4 | G: AP=3 <200>,
 (x: 7, y: 6) Edge Count: 3 | G: AP=3 <200>]

In [107]:
graph[(7, 3)]

(x: 7, y: 4) Edge Count: 3

In [106]:
round_number

3

In [97]:
graph

{(x: 1, y: 1) Edge Count: 3,
 (x: 2, y: 1) Edge Count: 19 | G: AP=3 <200>,
 (x: 3, y: 1) Edge Count: 3,
 (x: 4, y: 1) Edge Count: 3 | G: AP=3 <200>,
 (x: 5, y: 1) Edge Count: 3,
 (x: 6, y: 1) Edge Count: 3,
 (x: 7, y: 1) Edge Count: 2 | G: AP=3 <200>,
 (x: 1, y: 2) Edge Count: 3,
 (x: 2, y: 2) Edge Count: 4,
 (x: 3, y: 2) Edge Count: 4,
 (x: 4, y: 2) Edge Count: 4,
 (x: 5, y: 2) Edge Count: 4,
 (x: 6, y: 2) Edge Count: 4,
 (x: 7, y: 2) Edge Count: 3,
 (x: 1, y: 3) Edge Count: 3,
 (x: 2, y: 3) Edge Count: 4,
 (x: 3, y: 3) Edge Count: 4,
 (x: 4, y: 3) Edge Count: 4,
 (x: 5, y: 3) Edge Count: 4,
 (x: 6, y: 3) Edge Count: 4,
 (x: 7, y: 3) Edge Count: 3,
 (x: 1, y: 4) Edge Count: 3 | G: AP=3 <200>,
 (x: 2, y: 4) Edge Count: 4,
 (x: 3, y: 4) Edge Count: 4,
 (x: 4, y: 4) Edge Count: 4 | E: AP=3 <200>,
 (x: 5, y: 4) Edge Count: 4,
 (x: 6, y: 4) Edge Count: 4,
 (x: 7, y: 4) Edge Count: 3 | G: AP=3 <200>,
 (x: 1, y: 5) Edge Count: 3,
 (x: 2, y: 5) Edge Count: 4,
 (x: 3, y: 5) Edge Count: 4,
 (x:

In [84]:
print(test)

#########
#G..G..G#
#.......#
#.......#
#G..E..G#
#.......#
#.......#
#G..G..G#
#########


In [85]:
creatures

[(x: 1, y: 1) Edge Count: 2 | G: AP=3 <200>,
 (x: 4, y: 1) Edge Count: 3 | G: AP=3 <200>,
 (x: 7, y: 1) Edge Count: 2 | G: AP=3 <200>,
 (x: 1, y: 4) Edge Count: 3 | G: AP=3 <200>,
 (x: 4, y: 4) Edge Count: 4 | E: AP=3 <200>,
 (x: 7, y: 4) Edge Count: 3 | G: AP=3 <200>,
 (x: 1, y: 7) Edge Count: 2 | G: AP=3 <200>,
 (x: 4, y: 7) Edge Count: 3 | G: AP=3 <200>,
 (x: 7, y: 7) Edge Count: 2 | G: AP=3 <200>]

In [86]:
graph

{(x: 1, y: 1) Edge Count: 2 | G: AP=3 <200>,
 (x: 2, y: 1) Edge Count: 3,
 (x: 3, y: 1) Edge Count: 3,
 (x: 4, y: 1) Edge Count: 3 | G: AP=3 <200>,
 (x: 5, y: 1) Edge Count: 3,
 (x: 6, y: 1) Edge Count: 3,
 (x: 7, y: 1) Edge Count: 2 | G: AP=3 <200>,
 (x: 1, y: 2) Edge Count: 3,
 (x: 2, y: 2) Edge Count: 4,
 (x: 3, y: 2) Edge Count: 4,
 (x: 4, y: 2) Edge Count: 4,
 (x: 5, y: 2) Edge Count: 4,
 (x: 6, y: 2) Edge Count: 4,
 (x: 7, y: 2) Edge Count: 3,
 (x: 1, y: 3) Edge Count: 3,
 (x: 2, y: 3) Edge Count: 4,
 (x: 3, y: 3) Edge Count: 4,
 (x: 4, y: 3) Edge Count: 4,
 (x: 5, y: 3) Edge Count: 4,
 (x: 6, y: 3) Edge Count: 4,
 (x: 7, y: 3) Edge Count: 3,
 (x: 1, y: 4) Edge Count: 3 | G: AP=3 <200>,
 (x: 2, y: 4) Edge Count: 4,
 (x: 3, y: 4) Edge Count: 4,
 (x: 4, y: 4) Edge Count: 4 | E: AP=3 <200>,
 (x: 5, y: 4) Edge Count: 4,
 (x: 6, y: 4) Edge Count: 4,
 (x: 7, y: 4) Edge Count: 3 | G: AP=3 <200>,
 (x: 1, y: 5) Edge Count: 3,
 (x: 2, y: 5) Edge Count: 4,
 (x: 3, y: 5) Edge Count: 4,
 (x: 